# Gradio

https://www.gradio.app/guides/quickstart - быстрый старт

https://www.gradio.app/playground - набор интерактивный примеров 

В 2023 году API пакета изменился, поэтому многие старые примеры не работают. Рекомендуется обращаться к документации или к новым статьям.


####  Hello, World!

Принцип построения программы

<img src="https://github.com/gradio-app/gradio/blob/main/guides/assets/dataflow.svg?raw=true" width=1150>

In [ ]:
import gradio

# обработчик формы
# параметры - это данные полей (или др. элементы ввода) на форме
def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

# объект demo - форма
demo = gradio.Interface(
    fn=greet,                       # обработчик
    inputs=["text", "slider"],      # элементы ввода (поле ввода и слайдер)
    outputs=["text"],               # элементы вывода
    theme=gradio.themes.Base()
)
# https://www.gradio.app/guides/theming-guide   - настройка темы
    # gr.themes.Base()
    # gr.themes.Default()
    # gr.themes.Glass()
    # gr.themes.Monochrome()
    # gr.themes.Soft()


demo.launch()

### Некоторые элементы интерфейса

- `"text"`, `gradio.Textbox(···)`
- `"number"`, `gradio.Number(···)`
- `gradio.Label(···)`
- `gradio.Checkbox(···)`
- `gradio.Dropdown(···)`
- `gradio.Image(···)`
- `gradio.ImageEditor(···)` - примитивный редактор изображений, на котором можно (в том числе) рисовать
- `gradio.Sketchpad()` - панель (холст) для рисования
- `gradio.Dataframe(···)` - показывает датафрейм, может принимать данные для загрузки
- ``
- ``

### Пример - вывод изображения

#### Подготовка данных и модели

In [ ]:
# подготовка данных для примеры
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn
import pandas as pd

import gradio

data = load_iris()
X, y = data['data'], data['target']
TargetNames = data['target_names']

Data = pd.DataFrame(X)
Data.columns = data.feature_names
Data['class'] = y
Data['label'] = Data['class'].map({0:'setosa', 1:'versicolor', 2:'virginica'})

Forest = RandomForestClassifier().fit(X,y)

In [3]:
Data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class,label
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,virginica
146,6.3,2.5,5.0,1.9,2,virginica
147,6.5,3.0,5.2,2.0,2,virginica
148,6.2,3.4,5.4,2.3,2,virginica


#### Интерфейс с выводом изображения

In [7]:
def make_scatterplot(Data:pd.DataFrame, new_sw:float, new_pw: float):
    """Строит диаграмму рассеяния, добавляет новую точку в виде крестика"""
    fig = plt.figure( figsize=(5,5) )
    seaborn.scatterplot( data=Data[['sepal width (cm)', 'petal width (cm)', 'label']], x='sepal width (cm)', y='petal width (cm)', hue='label')
    # добавить ещё один маркер: размером 50, красного цвета
    plt.scatter(x=new_sw, y=new_pw, s=50, marker='x', color='r')
    # fig.show()
    plt.savefig('plot.png')
    return plt.imread('plot.png')

In [71]:
def predict(sl, sw, pl, pw: float):
    """Обработчик для события отправки данных формы;
    параметры - данные формы"""
    global Forest, Data

    y_pred = Forest.predict([[sl, sw, pl, pw]])[0]     # модель sklearn принимает список объектов, поэтом тут второй список
    return make_scatterplot(Data, sw, pw), TargetNames[y_pred]
    # return должен соответствовать выходным элементам (outputs) на форме


seaborn.set_style("darkgrid")

# создание формы
iris_form2 = gradio.Interface(
    predict,    # обработчик
    # элементы интерфейса для ввода данных    
    inputs= [ gradio.Number(label="длина чашелистика (sepal length)"),  gradio.Number(label="ширина чашелистика (sepal width)"), 
              gradio.Number(label="длина лепестка (petal length)"),     gradio.Number(label="ширина лепестка (petal width)")],
    # элементы интерфейса для вывода данных
    outputs=[gradio.Image(height=400, width=400), "text"]
)


# запуск сервера
iris_form2.launch()

Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


#### Интерфейс со столбчатой диаграммой, обратной связью (flag)

In [12]:
def predict2(sl, sw, pl, pw: float):
    """Обработчик для события отправки данных формы;
    параметры - данные формы"""
    global Forest, Data

    y_pred = Forest.predict_proba([[sl, sw, pl, pw]])[0]     # модель sklearn принимает список объектов, поэтом тут второй список
    result = pd.DataFrame( {'класс': TargetNames, 'вероятность': y_pred} )
    print(result)
    return make_scatterplot(Data, sw, pw), result
    # return должен соответствовать выходным элементам (outputs) на форме

seaborn.set_style("darkgrid")

# создание формы
iris_form2 = gradio.Interface(
    predict2,    # обработчик
    # элементы интерфейса для ввода данных    
    inputs= [ gradio.Number(label="длина чашелистика (sepal length)"),  gradio.Number(label="ширина чашелистика (sepal width)"), 
              gradio.Number(label="длина лепестка (petal length)"),     gradio.Number(label="ширина лепестка (petal width)")],
    # элементы интерфейса для вывода данных
    outputs=[gradio.Image(height=400, width=400), 
             gradio.BarPlot(title = "Вероятности классов", x='класс', y='вероятность', vertical=False,  interactive=True, height=100)],
    # flagging_callback = специальный объект для логирования, например gradio.flagging.CSVLogger
)


# запуск сервера
iris_form2.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


## Пример с вводом изображения

In [1]:
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier

# Датасет с рукописнымми цифрами
DataDigits = fetch_openml("mnist_784", version=1)
X = DataDigits['data'].values      # чб. изображения 28*28, но записанные в одномерный массив
y = DataDigits['target']

Forest2 = RandomForestClassifier(verbose=1).fit(X,y)


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:   19.9s


In [3]:
Forest2.predict( [np.random.randint(0, 255, 784)] )

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


array(['8'], dtype=object)

In [ ]:
import gradio

def predict3(img:np.array):
    """Обработчик для события отправки данных формы;
    параметры - данные формы"""
    global Forest2
    img = img['layers'][0].resize((28,28))
    img = np. array(img)
    img = img.reshape(-1)
    return Forest2.predict( [img] )[0]


# создание формы
mnist_form = gradio.Interface(
    predict3,    # обработчик
    # элементы интерфейса для ввода данных    
    inputs= [ gradio.Sketchpad( image_mode="L", type="pil", brush=gradio.Brush(colors=["#ffffff"]), crop_size=(300,300), width=300, height=300, canvas_size=(280,280))],
    # элементы интерфейса для вывода данных
    outputs=["text"],
    # flagging_callback = специальный объект для логирования, например gradio.flagging.CSVLogger
)


# запуск сервера
mnist_form.launch(max_threads=1)